In [1]:
!pip install pyswarm

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyswarm: filename=pyswarm-0.6-py3-none-any.whl size=4518 sha256=a2b2f4dff1a574376f7549415f0808577ccdd0e3cc7db46a64ceb36813f1b721
  Stored in directory: /root/.cache/pip/wheels/93/15/89/3970ef96abd6123028010a90f007c4e6a2bed700db0aa2d36a
Successfully built pyswarm


In [2]:
# Data manipulation and preprocessing
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
# Enables inline plotting in Jupyter/Colab notebooks
%matplotlib inline
np.set_printoptions(suppress=True)

# Scaling and model evaluation
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import silhouette_score, calinski_harabasz_score

# Machine learning models and tools
from sklearn.decomposition import PCA
from sklearn import cluster
from sklearn.cluster import DBSCAN, KMeans

##=== Pandas ==
import pandas as pd

# === Imports ===
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score

In [3]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')  # Only needed if you're loading data from Google Drive

Mounted at /content/drive


In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [5]:
Data= pd.read_csv("/content/drive/MyDrive/Network Maintenace - With & without maintenace/Original-data/2017-2020 (no main).csv")

In [6]:
Data.shape

(6277, 22)

In [7]:
Data.head(10)

AADT_mean_x  AADT_Single_Unit_mean_x  AADT_Combination_mean_x  \
0         4498                    502.0                    173.0   
1         4541                    390.0                    123.0   
2        40961                    820.0                   2295.0   
3         7826                    125.0                    697.0   
4        16157                    291.0                    226.0   
5        16570                    298.0                    232.0   
6        30297                    576.0                    121.0   
7        23284                    745.0                    256.0   
8        12370                    136.0                     74.0   
9        14380                    992.0                    316.0   

   Future_AADT_mean_x  IRI_mean_x  Thickness_Rigid_mean_x  \
0              5550.0        67.0                     0.0   
1              6560.0       138.0                     8.0   
2             40924.0        64.0                    11.5   
3              7837.0        96.0                     8.0   
4             32320.0       175.0                     0.0   
5             25186.0       163.0                     0.0   
6             42420.0       154.9                     0.0   
7             32600.0       118.7                     0.0   
8             17320.0       164.5                     0.0   
9             20130.0        88.0                     0.0   

   Thickness_Flexible_mean_x  Base_Thickness_mean_x  F_System_mode  \
0                        6.0                   15.0              4   
1                        4.4                   15.0              4   
2                        0.0                   15.0              1   
3                        0.0                   16.0              2   
4                        3.0                    8.0              3   
5                        3.0                    8.0              3   
6                        3.0                    8.0              3   
7                        3.0                    8.0              3   
8                        3.0                    8.0              3   
9                        3.0                    6.0              3   

    Urban_Type  Surface_Type_mode  Base_Type_mode_x  Rutting_mean_x  \
0  small urban                  6                 2             0.0   
1        urban                  7                 2             0.1   
2        urban                  3                 2             0.0   
3        rural                  3                 2             0.0   
4        urban                  2                 3             0.2   
5        urban                  2                 3             0.1   
6        urban                  2                 3             0.1   
7        urban                  2                 3             0.0   
8        urban                  2                 3             0.1   
9        urban                  2                 3             0.1   

   Cracking_Percent_mean_x  Faulting_mean_x  Last_Overlay_Thickness_mean_x  \
0                      0.0              0.0                            2.0   
1                      2.0              0.0                            2.0   
2                      0.0              0.0                            0.0   
3                    100.0              0.1                            0.0   
4                      0.0              0.0                            1.5   
5                      0.0              0.0                            1.5   
6                      0.4              0.0                            1.5   
7                      0.2              0.0                            1.5   
8                      0.0              0.0                            1.5   
9                      0.0              0.0                            1.5   

   IRI_mean_y  RHU_AV_x  FRZ_IDX_x  TEMP_AVG_x  PRECIPITATION_x  Age_x  
0        84.0      75.4      798.1         7.7           1107.3     24  
1       198.0      75.9 

In [8]:
Data['Urban_Type'].unique()

array(['small urban', 'urban', 'rural'], dtype=object)

In [9]:
# Filter rows where 'Urban_Code_mode' is 'rural'
rural_rows = Data[Data['Urban_Type'] == 'rural']

# Display the filtered DataFrame
rural_rows.count()

AADT_mean_x                      2598
AADT_Single_Unit_mean_x          2598
AADT_Combination_mean_x          2598
Future_AADT_mean_x               2598
IRI_mean_x                       2598
Thickness_Rigid_mean_x           2598
Thickness_Flexible_mean_x        2598
Base_Thickness_mean_x            2598
F_System_mode                    2598
Urban_Type                       2598
Surface_Type_mode                2598
Base_Type_mode_x                 2598
Rutting_mean_x                   2598
Cracking_Percent_mean_x          2598
Faulting_mean_x                  2598
Last_Overlay_Thickness_mean_x    2598
IRI_mean_y                       2598
RHU_AV_x                         2597
FRZ_IDX_x                        2597
TEMP_AVG_x                       2597
PRECIPITATION_x                  2597
Age_x                            2598
dtype: int64

In [10]:
categorical_columns = Data[['Urban_Type']]

encoder = OneHotEncoder(sparse_output=False)

one_hot_encoded = encoder.fit_transform(categorical_columns)

one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(['Urban_Type']))

df_encoded = pd.concat([Data, one_hot_df], axis=1)

df_encoded_2017_2020 = df_encoded.drop(categorical_columns, axis=1) ## Change this name to df_encoded_2017_2020

In [11]:
df_encoded_2017_2020.head(10)

AADT_mean_x  AADT_Single_Unit_mean_x  AADT_Combination_mean_x  \
0         4498                    502.0                    173.0   
1         4541                    390.0                    123.0   
2        40961                    820.0                   2295.0   
3         7826                    125.0                    697.0   
4        16157                    291.0                    226.0   
5        16570                    298.0                    232.0   
6        30297                    576.0                    121.0   
7        23284                    745.0                    256.0   
8        12370                    136.0                     74.0   
9        14380                    992.0                    316.0   

   Future_AADT_mean_x  IRI_mean_x  Thickness_Rigid_mean_x  \
0              5550.0        67.0                     0.0   
1              6560.0       138.0                     8.0   
2             40924.0        64.0                    11.5   
3              7837.0        96.0                     8.0   
4             32320.0       175.0                     0.0   
5             25186.0       163.0                     0.0   
6             42420.0       154.9                     0.0   
7             32600.0       118.7                     0.0   
8             17320.0       164.5                     0.0   
9             20130.0        88.0                     0.0   

   Thickness_Flexible_mean_x  Base_Thickness_mean_x  F_System_mode  \
0                        6.0                   15.0              4   
1                        4.4                   15.0              4   
2                        0.0                   15.0              1   
3                        0.0                   16.0              2   
4                        3.0                    8.0              3   
5                        3.0                    8.0              3   
6                        3.0                    8.0              3   
7                        3.0                    8.0              3   
8                        3.0                    8.0              3   
9                        3.0                    6.0              3   

   Surface_Type_mode  Base_Type_mode_x  Rutting_mean_x  \
0                  6                 2             0.0   
1                  7                 2             0.1   
2                  3                 2             0.0   
3                  3                 2             0.0   
4                  2                 3             0.2   
5                  2                 3             0.1   
6                  2                 3             0.1   
7                  2                 3             0.0   
8                  2                 3             0.1   
9                  2                 3             0.1   

   Cracking_Percent_mean_x  Faulting_mean_x  Last_Overlay_Thickness_mean_x  \
0                      0.0              0.0                            2.0   
1                      2.0              0.0                            2.0   
2                      0.0              0.0                            0.0   
3                    100.0              0.1                            0.0   
4                      0.0              0.0                            1.5   
5                      0.0              0.0                            1.5   
6                      0.4              0.0                            1.5   
7                      0.2              0.0                            1.5   
8                      0.0              0.0                            1.5   
9                      0.0              0.0                            1.5   

   IRI_mean_y  RHU_AV_x  FRZ_IDX_x  TEMP_AVG_x  PRECIPITATION_x  Age_x  \
0        84.0      75.4      798.1         7.7           1107.3     24   
1       198.0      75.9      891.6         7.4            986.4     16   
2        71.5      76.2      535.4         8.5           1068.6      5   
3       118.5    

In [12]:
df_encoded_2017_2020.columns

Index(['AADT_mean_x', 'AADT_Single_Unit_mean_x', 'AADT_Combination_mean_x',
       'Future_AADT_mean_x', 'IRI_mean_x', 'Thickness_Rigid_mean_x',
       'Thickness_Flexible_mean_x', 'Base_Thickness_mean_x', 'F_System_mode',
       'Surface_Type_mode', 'Base_Type_mode_x', 'Rutting_mean_x',
       'Cracking_Percent_mean_x', 'Faulting_mean_x',
       'Last_Overlay_Thickness_mean_x', 'IRI_mean_y', 'RHU_AV_x', 'FRZ_IDX_x',
       'TEMP_AVG_x', 'PRECIPITATION_x', 'Age_x', 'Urban_Type_rural',
       'Urban_Type_small urban', 'Urban_Type_urban'],
      dtype='object')

In [13]:
df_encoded_2017_2020.drop(['Last_Overlay_Thickness_mean_x'], axis=1, inplace=True)

In [14]:
df_encoded_2017_2020.columns

Index(['AADT_mean_x', 'AADT_Single_Unit_mean_x', 'AADT_Combination_mean_x',
       'Future_AADT_mean_x', 'IRI_mean_x', 'Thickness_Rigid_mean_x',
       'Thickness_Flexible_mean_x', 'Base_Thickness_mean_x', 'F_System_mode',
       'Surface_Type_mode', 'Base_Type_mode_x', 'Rutting_mean_x',
       'Cracking_Percent_mean_x', 'Faulting_mean_x', 'IRI_mean_y', 'RHU_AV_x',
       'FRZ_IDX_x', 'TEMP_AVG_x', 'PRECIPITATION_x', 'Age_x',
       'Urban_Type_rural', 'Urban_Type_small urban', 'Urban_Type_urban'],
      dtype='object')

In [15]:
df_encoded_2017_2020.head(10)

AADT_mean_x  AADT_Single_Unit_mean_x  AADT_Combination_mean_x  \
0         4498                    502.0                    173.0   
1         4541                    390.0                    123.0   
2        40961                    820.0                   2295.0   
3         7826                    125.0                    697.0   
4        16157                    291.0                    226.0   
5        16570                    298.0                    232.0   
6        30297                    576.0                    121.0   
7        23284                    745.0                    256.0   
8        12370                    136.0                     74.0   
9        14380                    992.0                    316.0   

   Future_AADT_mean_x  IRI_mean_x  Thickness_Rigid_mean_x  \
0              5550.0        67.0                     0.0   
1              6560.0       138.0                     8.0   
2             40924.0        64.0                    11.5   
3              7837.0        96.0                     8.0   
4             32320.0       175.0                     0.0   
5             25186.0       163.0                     0.0   
6             42420.0       154.9                     0.0   
7             32600.0       118.7                     0.0   
8             17320.0       164.5                     0.0   
9             20130.0        88.0                     0.0   

   Thickness_Flexible_mean_x  Base_Thickness_mean_x  F_System_mode  \
0                        6.0                   15.0              4   
1                        4.4                   15.0              4   
2                        0.0                   15.0              1   
3                        0.0                   16.0              2   
4                        3.0                    8.0              3   
5                        3.0                    8.0              3   
6                        3.0                    8.0              3   
7                        3.0                    8.0              3   
8                        3.0                    8.0              3   
9                        3.0                    6.0              3   

   Surface_Type_mode  Base_Type_mode_x  Rutting_mean_x  \
0                  6                 2             0.0   
1                  7                 2             0.1   
2                  3                 2             0.0   
3                  3                 2             0.0   
4                  2                 3             0.2   
5                  2                 3             0.1   
6                  2                 3             0.1   
7                  2                 3             0.0   
8                  2                 3             0.1   
9                  2                 3             0.1   

   Cracking_Percent_mean_x  Faulting_mean_x  IRI_mean_y  RHU_AV_x  FRZ_IDX_x  \
0                      0.0              0.0        84.0      75.4      798.1   
1                      2.0              0.0       198.0      75.9      891.6   
2                      0.0              0.0        71.5      76.2      535.4   
3                    100.0              0.1       118.5      76.8      618.1   
4                      0.0              0.0       200.0      67.1        3.7   
5                      0.0              0.0       191.5      62.8        0.0   
6                      0.4              0.0       157.2      39.5        3.1   
7                      0.2              0.0       128.0      39.5        3.1   
8                      0.0              0.0       165.4      39.5        3.1   
9                      0.0              0.0        96.8      68.9        2.9   

   TEMP_AVG_x  PRECIPITATION_x  Age_x  Urban_Type_rural  \
0         7.7           1107.3     24               0.0   
1         7.4            986.4     16               0.0   
2         8.5           1068.6      5               0.0   
3         7.8           1006.0      2               1.0

In [16]:
df_encoded_2017_2020.shape

(6277, 23)

In [17]:
import os

# Define the directory path
dir_path = "/content/drive/MyDrive/Network Maintenace - With & without maintenace/conference-encoded-data/No-main-encoded-dataset-2017-2020"

# Create the directory if it doesn't exist
os.makedirs(dir_path, exist_ok=True)

# Now save the file
df_encoded_2017_2020.to_csv(f"{dir_path}/df_encoded_no-main_2017_2020.csv", index=False)
